In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd 
import sys, os
from src import *

### KERNEL CONFIGURATION
Set up an application to be accelerated by loading the different kernel configurations into the kernel memory

In [2]:
# Load an existing kernel into memory
kmem_pos = 1
kmem_word = 0x802b

kmem = KER_CONF()
kmem.set_kernel_word(kmem_word, kmem_pos)
kmem.get_kernel_info(kmem_pos)

This kernel uses 43 instruction words starting at IMEM address 0.
It uses column(s): 0.
The SRF is located in SPM bank 0.


In [3]:
# Create a new kernel
kmem_pos = 2
kmem.set_kernel_params(num_instructions=20, imem_add_start=44, column_usage=3, srf_spm_addres=1, pos=kmem_pos)
print("Hex representation: " + kmem.get_word_in_hex(kmem_pos))
kmem.get_kernel_info(kmem_pos)

Hex representation: 0x38b14
This kernel uses 20 instruction words starting at IMEM address 44.
It uses column(s): both.
The SRF is located in SPM bank 1.


### Loop Control Unit IMEM 

In [4]:
# Load an existing imem word and decode it
imem_pos = 1
imem_word = 0xe5a08

lcu_imem = LCU_IMEM()
lcu_imem.set_kernel_word(imem_word, imem_pos)
lcu_imem.get_instruction_info(imem_pos)

Immediate value: 8
LCU is in loop control mode
Jump to address IMM + R1
No LCU registers are being written


In [5]:
# Create a new instruction
imem_pos = 2
lcu_imem.set_kernel_params(imm=3, rf_wsel=2, rf_we=1, alu_op=LCU_ALU_OPS.SADD, br_mode=0, muxb_sel=LCU_MUXB_SEL.ONE, muxa_sel=LCU_MUXA_SEL.ZERO, pos=imem_pos)
print("Hex representation: " + lcu_imem.get_word_in_hex(imem_pos))
lcu_imem.get_instruction_info(imem_pos)

Hex representation: 0xdc383
Immediate value: 3
LCU is in loop control mode
Performing ALU operation SADD between operands ZERO and ONE
Writing ALU result to LCU register 2


### Load Store Unit IMEM 

In [6]:
# Load an existing imem word and decode it
imem_pos = 1
imem_word = 0x539

lsu_imem = LSU_IMEM()
lsu_imem.set_kernel_word(imem_word, imem_pos)
lsu_imem.get_instruction_info(imem_pos)

No loading, storing, or shuffling taking place
Performing ALU operation SADD between operands R0 and ONE
Writing ALU result to LSU register 1


In [7]:
# Create a new instruction
imem_pos = 2
lsu_imem.set_kernel_params(rf_wsel=2, rf_we=1, alu_op=LSU_ALU_OPS.SRL.value, muxb_sel=LSU_MUXA_SEL.R5.value, muxa_sel=LSU_MUXB_SEL.TWO, vwr_shuf_op=SHUFFLE_SEL.CONCAT_BITREV_UPPER.value, vwr_shuf_sel=LSU_OP_MODE.SHUFFLE.value, pos=imem_pos)
print("Hex representation: " + lsu_imem.get_word_in_hex(imem_pos))
lsu_imem.get_instruction_info(imem_pos)

Hex representation: 0xe5aea
Shuffling VWR A and B data into VWR C using operation CONCAT_BITREV_UPPER
Performing ALU operation SRL between operands TWO and R5
Writing ALU result to LSU register 2


### Multiplexer Control Unit IMEM

In [8]:
# Load an existing imem word and decode it
imem_pos = 1
imem_word = 0x180

mxcu_imem = MXCU_IMEM()
mxcu_imem.set_kernel_word(imem_word, imem_pos)
mxcu_imem.get_instruction_info(imem_pos)

Not writing to VWRs
Reading from SRF index 6
Performing ALU operation NOP between operands R0 and R0
No MXCU registers are being written


In [9]:
# Create a new instruction
imem_pos = 2
vwr_row_we = [1, 1, 1, 0]
vwr_sel = MXCU_VWR_SEL.VWR_B.value
srf_sel = 3
alu_srf_write = ALU_SRF_WRITE.MXCU
srf_we = 1
rf_wsel = 0 
rf_we = 0 
alu_op =  MXCU_ALU_OPS.SADD
muxb_sel = MXCU_MUXB_SEL.R0
muxa_sel = MXCU_MUXA_SEL.TWO

mxcu_imem.set_kernel_params(vwr_row_we, vwr_sel, srf_sel, alu_srf_write, srf_we, rf_wsel, rf_we, alu_op, muxb_sel, muxa_sel, imem_pos)
mxcu_imem.get_instruction_info(imem_pos)

Writing to VWR rows [1 2 3] of VWR_B
Writing from MXCU ALU to SRF register 3
Performing ALU operation SADD between operands TWO and R0
No MXCU registers are being written


### Reconfigurable Cell IMEM

In [10]:
# Load an existing imem word and decode it
imem_pos = 1
imem_word = 0xe923

rc_imem = RC_IMEM()
rc_imem.set_kernel_word(imem_word, imem_pos)
rc_imem.get_instruction_info(imem_pos)

ALU is performing operations with 32-bit precision
Performing ALU operation LOR between operands SRF and ZERO
Writing ALU result to RC register 1


In [11]:
# Create a new instruction
imem_pos = 2
rf_wsel = 1 
rf_we = 1 
muxf_sel = RC_MUXF_SEL.RCT 
alu_op =  RC_ALU_OPS.INB_SF_INA
op_mode = 0 #Always keep this to zero; 16-bit mode is not supported yet
muxb_sel =  RC_MUXA_SEL.VWR_A
muxa_sel = RC_MUXA_SEL.VWR_B

rc_imem.set_kernel_params(rf_wsel, rf_we, muxf_sel, alu_op, op_mode, muxb_sel, muxa_sel, imem_pos)
rc_imem.get_instruction_info(imem_pos)

ALU is performing operations with 32-bit precision
Output VWR_B if sign flag of RCT == 1, else output VWR_A
Writing ALU result to RC register 1
